# This is my notebook that I will use for the capstone project in coursera

In [1]:
import pandas as pd
import numpy as np

### Getting the file information from wikipedia using pandas

In [2]:
file = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# returns a list of tables from the file
tables = pd.read_html(file)

### tables is a list of tables so how many tables does it contain?


In [3]:
print("# of tables: ", len(tables))

# of tables:  3


### combine each table into tables

In [4]:
df = pd.concat(tables,axis=1)
df.head()
df.columns

Index([  'Postal Code',       'Borough', 'Neighbourhood',               0,
                     1,               2,               3,               4,
                     5,               6,               7,               8,
                     9,              10,              11,              12,
                    13,              14,              15,              16,
                    17,               0,               1,               2,
                     3,               4,               5,               6,
                     7,               8,               9,              10,
                    11,              12,              13,              14,
                    15,              16,              17],
      dtype='object')

### df has alot of useless info in it.  Let's just specify postal cod, borough and neighourhood

In [5]:
# df has alot of useless info let's get rid of that
df_info = df[['Postal Code','Borough','Neighbourhood']]
df_info.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Let's drop boroughs that haven't been assigned

In [6]:
# Now let's drop rows that have "Not assigned" in the borough
df_info = df_info[ df['Borough'] != 'Not assigned']
df_info.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Postal codes should have their unique row with no duplicate rows
#### This cell shows that the number of unique rows is equal to the shape of the dataframe; i.e., there are no duplicate rows

In [7]:
# There should only exist one row for each Postal Code
uniqueRows = df_info['Postal Code'].nunique()

print(uniqueRows, df_info.shape)
print("Each row has unique postal code: ", uniqueRows == df_info.shape[0])

103 (103, 3)
Each row has unique postal code:  True


### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [8]:
df['Neighbourhood'].value_counts()

Not assigned                                                                                                     77
Downsview                                                                                                         4
Don Mills                                                                                                         2
Caledonia-Fairbanks                                                                                               1
Hillcrest Village                                                                                                 1
                                                                                                                 ..
Thorncliffe Park                                                                                                  1
Little Portugal, Trinity                                                                                          1
Stn A PO Boxes                                                          

### There's 77 'Not assigned' Neighbourhoods!  Let's change their neighbourhood to their respective borough!
#### I use value_counts() to check if "Not assigned" value still exists in the Neighbourhood column

In [9]:
# There's 77 Not assigned neighbourhoods!
for row in df_info.index:
    if(df_info.loc[row,'Neighbourhood'] == 'Not assigned'):
        df_info.loc[row,'Neighbourhood'] = df_info.loc[row, 'Borough']
        
df_info['Neighbourhood'].value_counts()

Downsview                                                                                                        4
Don Mills                                                                                                        2
The Beaches                                                                                                      1
Little Portugal, Trinity                                                                                         1
Brockton, Parkdale Village, Exhibition Place                                                                     1
                                                                                                                ..
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens    1
York Mills, Silver Hills                                                                                         1
Bayview Village                                                                 

#### How many rows x columns does the dataframe have?

In [10]:
# Shape of dataframe
df_info.shape

(103, 3)

### Importing coordinates from a csv file

In [15]:
geoDataURL = "https://cocl.us/Geospatial_data"
geoData = pd.read_csv(geoDataURL)
geoData.sort_values('Postal Code', inplace=True)
geoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Let's set the index to the postal code
#### This allows me to merge the correct lat & longitude to the correct row

In [16]:
geoData = geoData.set_index('Postal Code')

In [17]:
geoData.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


### Setting the index to postal code for the main dataframe

In [20]:
df_info = df_info.set_index('Postal Code')
df_info.head()

,Borough,Neighbourhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### This loops through the indicies (postal code) in the main df, gets the coordinates and assigns them to the corresponding column

In [21]:
for postal_code in df_info.index:
    lat = geoData.loc[postal_code, 'Latitude']
    long = geoData.loc[postal_code,'Longitude']
    df_info.loc[postal_code, 'Latitude'] = lat
    df_info.loc[postal_code, 'Longitude'] = long
    
df_info.head()

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### I might not want the indicies to be the postal code.  So the indicies are rest and the postal code is placed into its own column

In [22]:
df_info.reset_index(inplace=True)
df_info.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
